In [62]:
import torch
import os
import torch.nn as nn
import shutil
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torchsummary import summary 
import torch.optim as optim
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

veri setini birleştirme

In [19]:
def organize_data(base_dir, target_dir):
    # Create the target directories if they do not exist
    colour_dir = os.path.join(target_dir, 'colour')
    grey_dir = os.path.join(target_dir, 'grey')
    os.makedirs(colour_dir, exist_ok=True)
    os.makedirs(grey_dir, exist_ok=True)

    # Define the source directories
    cars_colour = os.path.join(base_dir, 'Cars', 'cars_colour')
    cars_gray = os.path.join(base_dir, 'Cars', 'cars_grey')
    flowers_colour = os.path.join(base_dir, 'Flowers', 'flowers_colour')
    flowers_gray = os.path.join(base_dir, 'Flowers', 'flowers_grey')

    # Copy colour images to the colour directory
    for src_dir in [cars_colour, flowers_colour]:
        for filename in os.listdir(src_dir):
            src_file = os.path.join(src_dir, filename)
            dst_file = os.path.join(colour_dir, filename)
            shutil.copyfile(src_file, dst_file)

    # Copy gray images to the gray directory
    for src_dir in [cars_gray, flowers_gray]:
        for filename in os.listdir(src_dir):
            src_file = os.path.join(src_dir, filename)
            dst_file = os.path.join(grey_dir, filename)
            shutil.copyfile(src_file, dst_file)

    print("Data organized successfully.")

# Base directory containing the 'cars' and 'flowers' directories
base_directory = '../../Downloads/data/'
target_dir = './data/'
organize_data(base_directory, target_dir)

Data organized successfully.


Veri setini train, val, test için bölme

In [20]:
def create_directories(base_dir):
    for subset in ['train', 'validation', 'test']:
        for category in ['colour', 'grey']:
            os.makedirs(os.path.join(base_dir, subset, category), exist_ok=True)

def split_and_copy_files(base_dir, filenames, train_ratio=0.7, validation_ratio=0.2):
    train_files, temp_files = train_test_split(filenames, test_size=(1 - train_ratio))
    validation_files, test_files = train_test_split(temp_files, test_size=(1 - validation_ratio / (1 - train_ratio)))

    return train_files, validation_files, test_files

def copy_files(file_list, src_dir, dst_dir):
    for filename in file_list:
        shutil.copyfile(os.path.join(src_dir, filename), os.path.join(dst_dir, filename))

def organize_data(base_dir):
    colour_dir = os.path.join(base_dir, 'colour')
    grey_dir = os.path.join(base_dir, 'grey')

    # List all filenames (assuming the filenames in colour and gray folders are identical)
    filenames = os.listdir(colour_dir)

    # Split the filenames into train, validation, and test sets
    train_files, validation_files, test_files = split_and_copy_files(base_dir, filenames)

    # Create the necessary directories
    create_directories(base_dir)

    # Copy files to the respective directories
    for subset, file_list in zip(['train', 'validation', 'test'], [train_files, validation_files, test_files]):
        copy_files(file_list, colour_dir, os.path.join(base_dir, subset, 'colour'))
        copy_files(file_list, grey_dir, os.path.join(base_dir, subset, 'grey'))

    print("Data organized successfully.")
    

# Base directory containing the 'colour' and 'gray' directories
base_directory = './data'
organize_data(base_directory)


Data organized successfully.


In [63]:
class ColorizationDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_pairs = []

        grey_dir = os.path.join(root_dir, "grey")
        color_dir = os.path.join(root_dir, "colour")

        grey_images = sorted(os.listdir(grey_dir))
        color_images = sorted(os.listdir(color_dir))

        for grey_img, color_img in zip(grey_images, color_images):
            self.image_pairs.append((os.path.join(grey_dir, grey_img), os.path.join(color_dir, color_img)))

    def __len__(self):
        return len(self.image_pairs)

    def __getitem__(self, idx):
        grey_img_path, color_img_path = self.image_pairs[idx]
        grey_image = Image.open(grey_img_path).convert('RGB')  # Gri tonlamalı görüntü RGB formatına dönüştürülür
        color_image = Image.open(color_img_path).convert('RGB')  # Renkli görüntü

        if self.transform:
            grey_image = self.transform(grey_image)
            color_image = self.transform(color_image)

        return grey_image, color_image

In [64]:
workspace_path = './data/'

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

train_dataset = ColorizationDataset(root_dir=os.path.join(workspace_path, "train"), transform=transform)
val_dataset = ColorizationDataset(root_dir=os.path.join(workspace_path, "validation"), transform=transform)
test_dataset = ColorizationDataset(root_dir=os.path.join(workspace_path, "test"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [67]:
class ColorizationNet(nn.Module):
    def __init__(self):
        super(ColorizationNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [72]:
def calculate_accuracy(outputs, targets, tolerance=0.1):
    difference = torch.abs(outputs - targets)
    correct = (difference < tolerance).float().mean()
    return correct

In [74]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ColorizationNet().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_accuracy = 0.0
    for grey_images, color_images in train_loader:
        grey_images, color_images = grey_images.to(device), color_images.to(device)
        
        optimizer.zero_grad()
        outputs = model(grey_images)
        loss = criterion(outputs, color_images)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * grey_images.size(0)
        train_accuracy += calculate_accuracy(outputs, color_images).item() * grey_images.size(0)
    
    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = train_accuracy / len(train_loader.dataset)
    
    model.eval()
    val_loss = 0.0
    val_accuracy = 0.0
    with torch.no_grad():
        for grey_images, color_images in val_loader:
            grey_images, color_images = grey_images.to(device), color_images.to(device)
            outputs = model(grey_images)
            loss = criterion(outputs, color_images)
            val_loss += loss.item() * grey_images.size(0)
            val_accuracy += calculate_accuracy(outputs, color_images).item() * grey_images.size(0)
    
    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = val_accuracy / len(val_loader.dataset)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

Epoch [1/50], Train Loss: 0.0670, Train Accuracy: 0.2586, Validation Loss: 0.0601, Validation Accuracy: 0.3037
Epoch [2/50], Train Loss: 0.0479, Train Accuracy: 0.3499, Validation Loss: 0.0575, Validation Accuracy: 0.2677
Epoch [3/50], Train Loss: 0.0390, Train Accuracy: 0.3716, Validation Loss: 0.0249, Validation Accuracy: 0.5717
Epoch [4/50], Train Loss: 0.0269, Train Accuracy: 0.5558, Validation Loss: 0.0283, Validation Accuracy: 0.5385
Epoch [5/50], Train Loss: 0.0262, Train Accuracy: 0.5474, Validation Loss: 0.0256, Validation Accuracy: 0.5554
Epoch [6/50], Train Loss: 0.0230, Train Accuracy: 0.5797, Validation Loss: 0.0232, Validation Accuracy: 0.5986
Epoch [7/50], Train Loss: 0.0206, Train Accuracy: 0.6309, Validation Loss: 0.0208, Validation Accuracy: 0.6275
Epoch [8/50], Train Loss: 0.0192, Train Accuracy: 0.6479, Validation Loss: 0.0197, Validation Accuracy: 0.6515
Epoch [9/50], Train Loss: 0.0182, Train Accuracy: 0.6800, Validation Loss: 0.0185, Validation Accuracy: 0.6751
E

In [77]:
model.eval()
test_loss = 0.0
test_accuracy = 0.0
with torch.no_grad():
    for grey_images, color_images in test_loader:
        grey_images, color_images = grey_images.to(device), color_images.to(device)
        outputs = model(grey_images)
        loss = criterion(outputs, color_images)
        test_loss += loss.item() * grey_images.size(0)
        test_accuracy += calculate_accuracy(outputs, color_images).item() * grey_images.size(0)

test_loss = test_loss / len(test_loader.dataset)
test_accuracy = test_accuracy / len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


Test Loss: 0.0098, Test Accuracy: 0.7990
